In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [2]:
from traj_xai.src import SimpleTransformerModel, TrajFormerModel

/opt/homebrew/anaconda3/envs/traj_xai/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/anaconda3/envs/traj_xai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/anaconda3/envs/traj_xai/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
from pactus import Dataset

## Segmentation


In [4]:
from traj_xai.src import (
    mdl_segmentation,
    rdp_segmentation,
    sliding_window_segmentation,
)

## Pertubation


In [5]:
from traj_xai.src import (
    gaussian_perturbation,
    scaling_perturbation,
    rotation_perturbation,
)

## Evaluation


## Experiment with all trajectories


In [ ]:
# LSTM Model
SEED = 0

# datasets = [
#     # Dataset.geolife(),
#     Dataset.animals(), x
#     # Dataset.hurdat2(), x
#     Dataset.uci_characters(), x
#     Dataset.cma_bst(), x
#     # Dataset.mnist_stroke(), 1k x
#     # Dataset.uci_pen_digits(), x
#     # Dataset.uci_gotrack(), x
#     Dataset.uci_movement_libras(),
# ]
# train, test = dataset.mnist_stroke(.8, random_state=SEED)

# Load dataset
dataset = Dataset.geolife()

# Split data into train and test subsets
train, test = dataset.split(0.98, random_state=SEED)
# train, test = dataset.cut(69000)

In [12]:
train.trajs[0].r

Vector([[-61.8,  26.7],
        [-62. ,  27.3],
        [-62. ,  27.9],
        [-61.7,  28.5],
        [-61.1,  29.3],
        [-60.2,  30.2],
        [-59. ,  31.5],
        [-57.4,  33.3],
        [-55.2,  35.6],
        [-52.5,  37.8],
        [-49.4,  39.8],
        [-46.5,  42.5],
        [-42. ,  46. ]])

In [ ]:
# train.classes

### DL Model


In [ ]:
max_length = 1000
embedding_size = 64

# Build and train the model
model = TrajFormerModel(
    c_out=len(dataset.classes),
)

# Train the model on the train dataset
model.train(train, dataset, epochs=10, batch_size=64)

# Evaluate the model on a test dataset
evaluation = model.evaluate(test)

# Print the evaluation
evaluation.show()

20:56:56 [INFO] Epoch 10/10, Loss: 35.6540
20:56:56 [INFO] Trained TrajFormer model with 1864 samples



General statistics:

Accuracy: 0.564
F1-score: 0.299
Mean precision: 0.309
Mean recall: 0.311

Confusion matrix:

0      1      2      3      4      5      precision
94.74  75.0   40.0   33.33  33.33  100.0  62.07  
5.26   25.0   20.0   33.33  0.0    0.0    40.0   
0.0    0.0    0.0    0.0    0.0    0.0    0.0    
0.0    0.0    0.0    33.33  33.33  0.0    50.0   
0.0    0.0    40.0   0.0    33.33  0.0    33.33  
0.0    0.0    0.0    0.0    0.0    0.0    0.0    
-------------------------------------------------
94.74  25.0   0.0    33.33  33.33  0.0    


### ML Model


In [ ]:
# # ML Model
# SEED = 0

# # datasets = [
# #     # Dataset.geolife(),
# #     Dataset.animals(),
# #     # Dataset.hurdat2(), -> RF cant work
# #     Dataset.cma_bst(),
# #     # Dataset.mnist_stroke(), -> okay
# #     # Dataset.uci_pen_digits(),
# #     # Dataset.uci_gotrack(),
# #     Dataset.uci_characters(),
# #     Dataset.uci_movement_libras(),-> okay
# # ]

# # Load dataset
# dataset = Dataset.geolife()

# # Split data into train and test subsets
# # train, test = dataset.split(.995, random_state=SEED)

# train, test = dataset.filter(
#         lambda traj, label: len(traj) >= 5
#         and traj.r.delta.norm.sum() > 0
#         and dataset.label_counts[label] > 5
#     ).split(
#         train_size=0.985,
#         random_state=SEED,
#     )

# # Build and train the model
# featurizer = featurizers.UniversalFeaturizer()
# model = XGBoostModel(featurizer=featurizer)
# # model = SVMModel(
# #     featurizer=featurizer,
# #     random_state=SEED,
# # )
# # DecisionTreeModel(
# #     featurizer=featurizer,
# #     max_depth=7,
# #     random_state=SEED,
# # # )


# # Train the model on the train dataset
# model.train(train, 5) # 5: CV

# # Evaluate the model on a test dataset
# evaluation = model.evaluate(test)

# # Print the evaluation
# evaluation.show()

### Experiment


In [ ]:
from traj_xai.src import run_experiments

In [ ]:
# segment_func = [rdp_segmentation]
# perturbation_func = [gaussian_perturbation]
segment_func = [sliding_window_segmentation]
perturbation_func = [scaling_perturbation, rotation_perturbation, gaussian_perturbation]

run_experiments(
    test,
    segment_func,
    perturbation_func,
    model,
)